## Verlet

In [1]:
import sys
sys.path.append("..")

import numpy as np
from system import system
from potential import D1
from integrator import D1_integrator

# Non-zero initial position with smaller mass and larger timestep
sys = system.D1(m=1.0, x=1.0, v=0.0, T=300.0, xi=1.0, dt=0.01, h=0.001)
pot = D1.Quadratic([1.0, 0.0])

# Run simulation
n_steps = 1000
positions = np.zeros(n_steps + 1)
velocities = np.zeros(n_steps)  # One less velocity than positions

# Store initial state
positions[0] = sys.x

# Integration loop
for i in range(n_steps):
    D1_integrator.verlet_step(sys, pot)
    # Store current state
    positions[i+1] = sys.x
    if i > 0:  # Store velocity from previous step
        velocities[i-1] = sys.v
    print(sys.x, sys.v)

# Calculate final velocity
velocities[-1] = (positions[-1] - positions[-2]) / sys.dt 

0.99995 0.0
0.999800005 -0.009999750000000418
0.9995500299994999 -0.019998500025003585
0.9992000999959999 -0.029995250200004486
0.9987502499825003 -0.03998900084998258
0.9982005249440025 -0.049978752599871834
0.9975509798530102 -0.05996350647450299
0.9968016796640327 -0.06994226399849035
0.9959526993070887 -0.07991402729607411
0.9950041236802141 -0.08987779919092942
0.9939560476409713 -0.09983258330586997
0.9928085759969645 -0.1097773841624794
0.9915618234953579 -0.11971120728067053
0.9902159148114018 -0.12963305927813362
0.9887709845359646 -0.139541947969668
0.9872271771620738 -0.14943688246640274
0.9855846470704668 -0.15931687327488953
0.9838435585141527 -0.16918093239605136
0.9820040856019873 -0.17902807342397575
0.9800664122812616 -0.188857311644558
0.9780307323193077 -0.19866766413397796
0.9758972492841219 -0.20845814985698263
0.9736661765240077 -0.218227789765002
0.971337737146241 -0.22797560689404306
0.9689121639947598 -0.23770062646239487
0.966389699626879 -0.2474018759681007
0

## Euler

In [3]:
# Non-zero initial position with smaller mass and larger timestep
sys = system.D1(m=1.0, x=1.0, v=0.0, T=300.0, xi=1.0, dt=0.01, h=0.001)
pot = D1.Quadratic([1.0, 0.0])

# Run simulation
n_steps = 1000
positions = np.zeros(n_steps + 1)
velocities = np.zeros(n_steps + 1)

# Store initial state
positions[0] = sys.x
velocities[0] = sys.v

# Integration loop
for i in range(n_steps):
    D1_integrator.euler_step(sys, pot)
    # Store current state
    positions[i+1] = sys.x
    velocities[i+1] = sys.v
    print(sys.x, sys.v)

0.99995 -0.01
0.9998000025 -0.0199995
0.999550017499875 -0.029997500025
0.99920006499875 -0.03999300019999875
0.9987501749935 -0.04998500084998625
0.9982003874762505 -0.05997250259992125
0.9975507524308774 -0.06995450647468375
0.9968013298285091 -0.07993001399899252
0.9959521896220277 -0.08989802729727761
0.9950034117395739 -0.09985754919349789
0.993955086077052 -0.10980758331089363
0.9928073124896392 -0.11974713417166415
0.991560200782298 -0.12967520729656054
0.9902138706992932 -0.13959080930438353
0.9887684519127145 -0.14949294801137647
0.987224084010005 -0.1593806325305036
0.9855809164804995 -0.16925287337060366
0.9838391087009694 -0.17910868253540865
0.9819988299201802 -0.18894707362241836
0.98006025924246 -0.19876706192162016
0.9780235856102817 -0.20856766451404476
0.9758890077858607 -0.2183479003701476
0.9736567343317699 -0.2281067904480062
0.9713269835905733 -0.2378433577913239
0.9688999836634806 -0.24755662762722963
0.9663759723880251 -0.2572456274638644
0.963755197314767 -0.26

## Leapfrog

In [5]:
# Initialize system
sys = system.D1(m=1.0, x=1.0, v=0.0, T=300.0, xi=1.0, dt=0.01, h=0.001)
pot = D1.Quadratic([1.0, 0.0])  # k=1.0, x0=0.0

# Run simulation with step-by-step leap-frog
n_steps = 1000
positions = np.zeros(n_steps + 1)
velocities = np.zeros(n_steps + 1)
velocity_halfsteps = np.zeros(n_steps + 2)  # +2 for initial and final half steps

# Store initial state
positions[0] = sys.x
velocities[0] = sys.v

# Calculate initial force
force = pot.force(positions[0], sys.h)[0]

# Initialize velocity half steps exactly as in leapfrog()
velocity_halfsteps[0] = sys.v - (force/sys.m) * (sys.dt/2)
velocity_halfsteps[1] = sys.v + (force/sys.m) * (sys.dt/2)

# Set the system's v_half to match
sys.v_half = velocity_halfsteps[1]

# Integration loop
for i in range(n_steps):
    # Store the current position
    x_current = sys.x
    
    # Calculate force at current position
    force = pot.force(x_current, sys.h)[0]
    
    # Update velocity at half step (t+1)
    velocity_halfsteps[i+1] = velocity_halfsteps[i] + (force/sys.m) * sys.dt
    
    # Update position using half-step velocity
    sys.x = x_current + velocity_halfsteps[i+1] * sys.dt
    
    # Calculate velocity at full step (for output)
    sys.v = (velocity_halfsteps[i+1] + velocity_halfsteps[i]) / 2
    
    # Store current state
    positions[i+1] = sys.x
    velocities[i+1] = sys.v
    print(sys.x, sys.v)

0.99995 0.0
0.999800005 -0.00999975
0.9995500299995 -0.019998500024999998
0.9992000999960001 -0.0299952501999975
0.9987502499825006 -0.039989000849975004
0.9982005249440028 -0.04997875259986751
0.9975509798530106 -0.05996350647450003
0.9968016796640331 -0.0699422639984851
0.9959526993070892 -0.07991402729607032
0.9950041236802146 -0.08987779919092592
0.993956047640972 -0.09983258330586245
0.9928085759969653 -0.10977738416246838
0.9915618234953588 -0.11971120728065807
0.9902159148114028 -0.1296330592781197
0.9887709845359657 -0.13954194796965352
0.987227177162075 -0.14943688246639036
0.9855846470704681 -0.15931687327488056
0.9838435585141542 -0.16918093239604326
0.9820040856019888 -0.17902807342396637
0.9800664122812632 -0.1888573116445471
0.9780307323193095 -0.19866766413396336
0.9758972492841238 -0.20845814985696626
0.9736661765240098 -0.21822778976498342
0.9713377371462434 -0.22797560689402407
0.9689121639947623 -0.23770062646237536
0.9663896996268818 -0.2474018759680804
0.9637705962

## Velocity Verlet

In [6]:
# Initialize system
sys = system.D1(m=1.0, x=1.0, v=0.0, T=300.0, xi=1.0, dt=0.01, h=0.001)
pot = D1.Quadratic([1.0, 0.0])  # k=1.0, x0=0.0

# Run simulation with step-by-step velocity Verlet
n_steps = 1000
positions = np.zeros(n_steps + 1)
velocities = np.zeros(n_steps + 1)

# Store initial state
positions[0] = sys.x
velocities[0] = sys.v

# Integration loop
for i in range(n_steps):
    D1_integrator.velocity_verlet_step(sys, pot)
    
    # Store current state
    positions[i+1] = sys.x
    velocities[i+1] = sys.v
    print(sys.x, sys.v)

0.99995 -0.00999975
0.999800005 -0.019998500025
0.9995500299995 -0.0299952501999975
0.9992000999960001 -0.039989000849975004
0.9987502499825006 -0.049978752599867504
0.9982005249440028 -0.05996350647450002
0.9975509798530106 -0.06994226399848509
0.996801679664033 -0.0799140272960703
0.9959526993070891 -0.08987779919092592
0.9950041236802144 -0.09983258330586244
0.9939560476409718 -0.10977738416246836
0.992808575996965 -0.11971120728065805
0.9915618234953586 -0.12963305927811966
0.9902159148114026 -0.13954194796965347
0.9887709845359655 -0.1494368824663903
0.9872271771620748 -0.1593168732748805
0.9855846470704678 -0.1691809323960432
0.9838435585141538 -0.17902807342396632
0.9820040856019884 -0.18885731164454703
0.9800664122812628 -0.19866766413396328
0.978030732319309 -0.20845814985696615
0.9758972492841234 -0.2182277897649833
0.9736661765240093 -0.22797560689402396
0.9713377371462428 -0.23770062646237522
0.9689121639947618 -0.24740187596808025
0.9663896996268813 -0.2570783852861885
0.9